In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
    
data_transform=transforms.Compose([  
    transforms.Resize((128,128)),   
    transforms.ToTensor()   
])

train_data=datasets.ImageFolder(root="data/train",transform=data_transform) 
test_data=datasets.ImageFolder(root="data/test",transform=data_transform)
valid_data=datasets.ImageFolder(root="data/validation",transform=data_transform)



In [ ]:
train_loader=DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=DataLoader(test_data,batch_size=32,shuffle=False,num_workers=2)
valid_loader=DataLoader(valid_data,batch_size=32,shuffle=False,num_workers=2)

In [4]:
class_names=train_data.classes
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn as nn

class WaferFault(nn.Module):
    def __init__(self, num_classes=9):
        super(WaferFault,self).__init__()

        self.conv1=nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2=nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3=nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        self.pool=nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten=nn.Flatten()
        self.fc1=nn.Linear(128*16*16,128)
        self.dropout=nn.Dropout(0.5)
        self.fc2=nn.Linear(128,num_classes)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(F.relu(self.conv3(x)))
        x=self.flatten(x)
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        x=self.fc2(x)
        return x


import torch.nn.functional as F
model=WaferFault(num_classes=9).to(device)

In [6]:
import torch.optim as optim

loss=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

epochs=15   
for epoch in range(epochs):
    model.train()
    running_loss=0

    for img,label in train_loader:
        img,label=img.to(device),label.to(device)

        optimizer.zero_grad()
        output=model(img)

        losses=loss(output,label)

        losses.backward()

        optimizer.step()
        running_loss+=losses.item()

    print(f"Epoch: {epoch+1}, loss: {running_loss/len(train_loader)}")



Epoch: 1, loss: 0.8465438069541598
Epoch: 2, loss: 0.6065998564570979
Epoch: 3, loss: 0.5003516548562571
Epoch: 4, loss: 0.41259426235989793
Epoch: 5, loss: 0.3463855379571517
Epoch: 6, loss: 0.3041037859018546
Epoch: 7, loss: 0.264862932478551
Epoch: 8, loss: 0.23438574016155278
Epoch: 9, loss: 0.21407966593005617
Epoch: 10, loss: 0.1887365279520755
Epoch: 11, loss: 0.171226106471092
Epoch: 12, loss: 0.15914351585679917
Epoch: 13, loss: 0.15156438578870607
Epoch: 14, loss: 0.13758158968374148
Epoch: 15, loss: 0.13144543973278622


Evaluation on test set

In [7]:
model.eval()
correct=0
tot=0

with torch.no_grad():
    for img,label in test_loader:
        img,label=img.to(device),label.to(device)
        output=model(img)
        _,predicted=torch.max(output,1)
        tot+=label.size(0)
        correct+=(predicted==label).sum().item()

print(f"Test data accuracy: {100*(correct/tot)}%")

Test data accuracy: 95.53941908713693%


Evaluation of Validation data

In [8]:
model.eval()
correct=0
tot=0

with torch.no_grad():
    for img,label in valid_loader:
        img,label=img.to(device),label.to(device)
        output=model(img)
        _,predicted=torch.max(output,1)
        tot+=label.size(0)
        correct+=(predicted==label).sum().item()

print(f"Validation data accuracy: {100*(correct/tot)}%")


Validation data accuracy: 95.53075727292915%


In [ ]:
torch.save(model.state_dict(), 'wafer_model.pth')
print("Model saved successfully!")

Model saved successfully!
